In [ ]:
import numpy as np
import math
import sys
import os
import csv
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.applications import resnet50
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense,Dropout,Flatten, BatchNormalization, Reshape, GlobalAveragePooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD,Adam
from keras.callbacks import History 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
def preprocess_data(X, Y):
  
  X = X / 255.0
  X_pp = resnet50.preprocess_input(X)
  Y_pp = tf.keras.utils.to_categorical(Y,10)

  return X_pp, Y_pp

In [ ]:
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [ ]:
x_train_part, x_test_part, y_train_part, y_test_part = train_test_split(x_train, y_train, test_size=0.3)

In [ ]:
x_train_part, y_train_part = preprocess_data(x_train_part, y_train_part)
x_test_part, y_test_part = preprocess_data(x_test_part, y_test_part)

In [ ]:
modelResNet = resnet50.ResNet50(
    include_top = True,
    weights=None,
    input_shape = (32, 32, 3),
    classes=10)

In [ ]:
def createModel(array_dimension):
  model = tf.keras.Sequential()

  model.add(modelResNet)

  for i in range(0, len(array_dimension)-1, 2):
    model.add(tf.keras.layers.Dense(array_dimension[i], activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())    

  model.add(tf.keras.layers.Dense(array_dimension[-1], activation="softmax"))
  model.add(tf.keras.layers.Dense(10*10, activation="relu"))
  model.add(tf.keras.layers.Reshape((10,10)))

  return model

In [ ]:
# def createModel(array_dimension):
  
  
#   for i in range(0, len(array_dimension), 2):
#     x = tf.keras.layers.Dense(array_dimension[i], activation="relu")(x)
#     x = tf.keras.layers.Dropout(0.5)(x)

#   x = tf.keras.layers.Dense(2048, activation="relu")(x)
#   x = tf.keras.layers.Dense(10, activation="softmax")(x)

#   model = Model(inputs = modelResNet.input, outputs = x)
#   model.summary()
  
#   return model

In [ ]:
def evaluate_accuracy(model, Xtrain, Ytrain, Xtest, Ytest, emax):
  model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics = ['accuracy'])

  flops = sum([np.prod(tf.keras.backend.get_value(w).shape) for w in model.trainable_weights])

  Xtrain, Ytrain = preprocess_data(Xtrain, Ytrain)
  Xtest, Ytest = preprocess_data(Xtest, Ytest)

  history = History()

  historyRes = model.fit(Xtrain, Ytrain, validation_data=(Xtest,Ytest), epochs=emax, batch_size=128, callbacks=[history])
  
  accuracy = historyRes.history.get('accuracy')[-1]

  return accuracy, flops

In [ ]:
def schwefel(array):
    sum = 0
    fitness = 0
    for x in array:
        sum = sum + x * np.sin(np.sqrt(np.abs(x)))
    fitness = 418.9829 * len(array) - sum
    return fitness

In [ ]:
def calculation(model, t):
    fitness = evaluate_accuracy(model, x_train_part, y_train_part, x_test_part, y_test_part, Config.get_epochs())
    return fitness 


In [ ]:
def eggholder(array):
    z = - (array[1] + 47) * np.sin(np.sqrt(abs(array[1] + (array[0]/2) +47))) - array[0] *np.sin(np.sqrt(abs(array[0] - (array[1]+47))))
    return z

In [ ]:
def sphere(array):
    fitness = 0
    for i in range(len(array)):
        fitness = fitness + array[i]**2
    return fitness

In [ ]:
def rastrigin(array):
    sum = 0
    fitness = 0
    for x in array:
        sum = sum + x**2 - 10 * np.cos(2 * np.pi * x)
    fitness = 10.0 * len(array) + sum
    return fitness

In [ ]:
def michalewicz(array):#for the number of Dimension is 2
    sum = 0
    fitness = 0
    m = 10
    for (i,x) in enumerate(array, start=1):
        sum = sum + np.sin(x) * np.sin((i * (x**2) )/np.pi)**(2*m)
    fitness = -sum
    return fitness

In [ ]:
class Config:
    __PopulationSize = 50 # Population Size
    __MaxDomain = 500 # variable upper limit
    __MinDomain = -500 # variable lower limit
    __Lambda = 1.5 # parameter for Levy flight
    __Pa = 0.25
    __Step_Size = 0.01
    __Dimension = 10 # The number of dimension
    __Trial = 31
    __Iteration = 10
    __Nblocks = 5
    __Epochs = 30
    __Vector_ranges = np.array([[4, 8], [4, 6], [5, 15], [4, 6] ,[12, 40], [4, 6] ,[30, 80], [4, 6], [60, 100], [4, 6]])

    @classmethod
    def get_nblocks(cls):
        return cls.__Nblocks

    @classmethod
    def get_epochs(cls):
        return cls.__Epochs

    @classmethod
    def get_vector_ranges(cls):
        return cls.__Vector_ranges

    @classmethod
    def set_nblocks(cls, _nblocks):
        cls.__Nblocks = _nblocks

    @classmethod
    def get_population_size(cls):
        return cls.__PopulationSize

    @classmethod
    def get_Pa(cls):
        return cls.__Pa

    @classmethod
    def get_iteration(cls):
        return cls.__Iteration

    @classmethod
    def get_trial(cls):
        return cls.__Trial

    @classmethod
    def get_dimension(cls):
        return cls.__Dimension

    @classmethod
    def get_max_domain(cls):
        return cls.__MaxDomain

    @classmethod
    def set_max_domain(cls, _max_domain):
        cls.__MaxDomain = _max_domain

    @classmethod
    def get_min_domain(cls):
        return cls.__MinDomain

    @classmethod
    def set_min_domain(cls, _min_domain):
        cls.__MinDomain = _min_domain

    @classmethod
    def get_lambda(cls):
        return cls.__Lambda

    @classmethod
    def set_lambda(cls, _lambda):
        cls.__Lambda = _lambda

    @classmethod
    def get_stepsize(cls):
        return cls.__Step_Size


In [ ]:
def levy_flight(Lambda):
    #generate step from levy distribution
    sigma1 = np.power((math.gamma(1 + Lambda) * np.sin((np.pi * Lambda) / 2)) \
                      / math.gamma((1 + Lambda) / 2) * np.power(2, (Lambda - 1) / 2), 1 / Lambda)
    sigma2 = 1
    u = np.random.normal(0, sigma1, size=Config.get_nblocks())
    v = np.random.normal(0, sigma2, size=Config.get_nblocks())
    step = u / np.power(np.fabs(v), 1 / Lambda)

    return step

In [ ]:
class Individual:
    def __init__(self):
        self.__position = np.random.rand(Config.get_dimension()) * (Config.get_max_domain() - Config.get_min_domain())  + Config.get_min_domain()
        self.__nBlocks = Config.get_nblocks()
        ranges = Config.get_vector_ranges()
        self.__layers = np.random.randint(ranges[:, 1]-ranges[:, 0]) + ranges[:, 0]
        self.__model = createModel(self.__layers)
        self.__fitness = calculation(self.__model,0) # iteration = 0

    def get_position(self):
        return self.__position

    def get_nblocks(self):
        return self.__nBlocks

    def set_position(self, position):
        self.__position = position

    def get_fitness(self):
        return self.__fitness

    def get_model(self):
        return self.__model

    def set_model(self, model):
        self.__model = model

    def set_fitness(self, fitness):
        self.__fitness = fitness

    def abandon(self):
        # abandon some variables
        for i in range(0, len(self.__layers), 2):
            p = np.random.rand()
            if p < Config.get_Pa():
                self.__layers[i] = np.random.rand() * (Config.get_vector_ranges[i][1] - Config.get_vector_ranges[i][0])  + Config.get_vector_ranges[i][0]

    def get_cuckoo(self):

        step_size = Config.get_stepsize() * levy_flight(Config.get_lambda())

        # Update position
        self.__layers[range(0, len(self.__layers), 2)] = self.__layers[range(0, len(self.__layers), 2)] + step_size

        # Simple Boundary Rule
        for i in range(0, len(self.__layers), 2):
            if self.__layers[i] > Config.get_vector_ranges()[i][1]:
                self.__layers[i] = Config.get_max_domain()[i][1]
            if self.__layers[i] < Config.get_min_domain()[i][0]:
                self.__layers[i] = Config.get_min_domain()[i][0]

    def print_info(self,i):
        print("ind:","{0:3d}".format(i),
              "|| fitness:",str(self.__fitness[0]).rjust(14," "), " | ", str(self.__fitness[1]))


In [ ]:
results_list = []

def main():
    for trial in range(Config.get_trial()):
        np.random.seed(trial)

        cs_list = []
        '''Generate Initial Population'''
        for p in range(Config.get_population_size()):
            cs_list.append(Individual())

        '''Sort List'''
        cs_list = sorted(cs_list, key=lambda x: x.get_fitness()[0])

        '''Find Initial Best'''
        BestModel = cs_list[0].get_model()
        BestFitness = calculation(cs_list[0].get_model(),0)

        '''Main Loop'''
        for iteration in range(Config.get_iteration()):

            '''Generate New Solutions'''
            for i in range(len(cs_list)):
                cs_list[i].get_cuckoo()
                cs_list[i].set_fitness(calculation(cs_list[i].get_model(),iteration))

                '''random choice (say j)'''
                j = np.random.randint(low=0, high=Config.get_population_size())
                while j == i: #random id[say j] ≠ i
                    j = np.random.randint(0, Config.get_population_size())

                # for minimize problem
                if(cs_list[i].get_fitness()[0] < cs_list[j].get_fitness()[0]):
                    cs_list[j].set_model(cs_list[i].get_model())
                    cs_list[j].set_fitness(cs_list[i].get_fitness())
                elif(cs_list[i].get_fitness()[0] == cs_list[j].get_fitness()[0] and
                        cs_list[i].get_fitness()[1] < cs_list[j].get_fitness()[1] ):
                    cs_list[j].set_model(cs_list[i].get_model())
                    cs_list[j].set_fitness(cs_list[i].get_fitness())

            '''Sort (to Keep Best)'''
            cs_list = sorted(cs_list, key=lambda x: x.get_fitness()[0])

            '''Abandon Solutions (exclude the best)'''
            for a in range(1,len(cs_list)):
                r = np.random.rand()
                if(r < Config.get_Pa()):
                    cs_list[a].abandon()
                    cs_list[a].set_fitness(calculation(cs_list[a].get_model(),iteration))

            '''Sort to Find the Best'''
            cs_list = sorted(cs_list, key=lambda x: x.get_fitness()[0])

            if cs_list[0].get_fitness()[0] < BestFitness[0]:
                BestFitness = cs_list[0].get_fitness()[0]
                BestModel = cs_list[0].get_model()

            sys.stdout.write("\r Trial:%3d , Iteration:%7d, BestFitness:%.4f" % (trial , iteration, BestFitness))

            results_list.append((BestModel, BestFitness))

        for i in results_list:
          i.summary()



main()

Epoch 1/30
274/274 [==============================] - 30s 87ms/step - loss: nan - accuracy: 0.5848 - val_loss: 3.3045 - val_accuracy: 0.7397
Epoch 2/30
274/274 [==============================] - 22s 81ms/step - loss: nan - accuracy: 0.7345 - val_loss: 3.1829 - val_accuracy: 0.7397
Epoch 3/30
274/274 [==============================] - 22s 80ms/step - loss: nan - accuracy: 0.7489 - val_loss: 3.1787 - val_accuracy: 0.7397
Epoch 4/30
274/274 [==============================] - 22s 81ms/step - loss: nan - accuracy: 0.7518 - val_loss: 3.1682 - val_accuracy: 0.7397
Epoch 5/30
274/274 [==============================] - 22s 81ms/step - loss: nan - accuracy: 0.7529 - val_loss: 3.1652 - val_accuracy: 0.7397
Epoch 6/30
274/274 [==============================] - 22s 81ms/step - loss: nan - accuracy: 0.7532 - val_loss: 3.1649 - val_accuracy: 0.7397
Epoch 7/30
274/274 [==============================] - 22s 80ms/step - loss: nan - accuracy: 0.7530 - val_loss: 3.1641 - val_accuracy: 0.7397
Epoch 8/30
27

In [ ]:
def show_acc(historyRes):
  plt.plot(historyRes.history["accuracy"])
  plt.plot(historyRes.history["val_accuracy"])
  plt.title("Accuracy CNN")
  plt.ylabel("Accuracy")
  plt.xlabel("Number of epochs")
  plt.legend(["training_data", "testing_data"])
  plt.figure()
  plt.title("Loss CNN")
  plt.ylabel("Loss")
  plt.plot(historyConv.history["loss"])
  plt.plot(historyConv.history["val_loss"])
  plt.xlabel("Number of epochs")
  plt.legend(["train", "test"])
  plt.show()